In [7]:
from bs4 import BeautifulSoup

# https://www.potatopro.com/product-types/potato-starch-and-derivates 들어가서 
# load more 모두 하고 html 소스 가져와서 처리함.
# html 소스 가져올때 개발자 도구 요소탭에서 html 우클릭 해서 전체 복사함.

with open('input/potato_pro.html', 'r+') as f:
    html = f.read()

soup = BeautifulSoup(html, 'html.parser')


In [14]:
import pytz
from datetime import datetime
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests.adapters import HTTPAdapter, Retry

requests.adapters.DEFAULT_POOLSIZE = 100
retries = Retry(total=20)
# 세션을 생성하고 ConnectionPool 크기를 설정
session = requests.Session()
session.mount('https://', requests.adapters.HTTPAdapter(pool_connections=100, pool_maxsize=100, max_retries=retries))

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}


In [8]:
a_list = soup.find_all('a', class_="main-link-wrap")

In [66]:
import json

def get_detail_page(sub_soup):   
    contact_info = {}
    contact_info["name"] = sub_soup.find('h1', class_="title page-title").text.strip()
    website_div = sub_soup.find('div', class_='mb-3 border-bottom-grey')
    if website_div.find('a'):
        contact_info['website_logo'] = website_div.find('a').get('href')

    for a in sub_soup.find('div', class_='contact-info').find_all('a', href=True):
        key = a['aria-label'].lower()
        if 'mailto:' in a['href']:
            value = a['href'].replace('mailto:', '')
        elif 'tel:' in a['href']:
            value = a['href'].replace('tel:', '').strip()
        else:
            value = a['href']
        contact_info[key] = value

    product_list = []
    for a in sub_soup.find('ul', class_='accordion_term').find_all('a'):
        product_list.append(a.text.strip())

    contact_info['products'] = json.dumps(product_list, ensure_ascii=False)
    contact_info['address'] = sub_soup.find('p', class_="address").text
    return contact_info

In [67]:
import time
from urllib.parse import urlparse, urljoin
main_url = 'https://www.potatopro.com'

info_list = []

for a in a_list:
    sub_url = urljoin(main_url, a.get('href'))
    sub_response = session.get(sub_url, headers=headers)
    sub_soup = BeautifulSoup(sub_response.text, 'html.parser')
    info = get_detail_page(sub_soup)
    info_list.append(info)
    time.sleep(0.5)


In [72]:

sub_url = urljoin(main_url, a_list[5].get('href'))
sub_url

'https://www.potatopro.com/companies/agrawest-foods'

In [68]:
import pandas as pd
df = pd.DataFrame(info_list)
df.to_csv('output/potatopro_20240207.csv', index=False)